In [1]:
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.metrics import mean_squared_error, r2_score
from math import sqrt
import numpy as np
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle
from math import ceil

In [3]:
# key between our document and PDS's
# they don't show Ti metal and only one Graphite spectrum
ct_key = {
    'NAu-2_Med-S':'NAU2MEDS',
    'NAu-2_Hi-S':'NAU2HIGHS',
    'NAu-2_Low-S':'NAU2LOWS',
    'Picrite':'PICRITE',
    'Norite':'NORITE',
    'Shergottite':'SHERGOTTITE',
    'Kga-d_Med-S':'KGA2MEDS',
    'Macusanite':'MACUSANITE'
}

# load data
df = pd.read_csv('P:\\CHEMCAM_from_PDS\\LIBS_CCS_metadata_w_moc.csv')
df = df[df.Target.isin(ct_key.keys())].reset_index(drop=True)

# add actual comps
comp_cols = [c.split(' ')[0] for c in df.columns if 'RMSEP' in c]

df['PELLET NAME'] = df.Target.map(ct_key)
# actual
comps = pd.read_excel('Z:\\Millennium Set\\Millennium_COMPS_viewonly.xlsx')
comps.columns = list(comps.iloc[0])
comps = comps.drop(index=[0,1])

# format
new_comp_cols = comp_cols.copy()
new_comp_cols.insert(0,'PELLET NAME')
new_comp_cols = ['FeO' if c == 'FeOT' else c for c in new_comp_cols]
comps = comps[new_comp_cols]

new_comp_cols = ['FeOT' if c == 'FeO' else c for c in new_comp_cols]
new_comp_cols = [c+'_actual' for c in comp_cols]
new_comp_cols.insert(0,'PELLET NAME')
comps.columns = new_comp_cols

# add actual values
df = df.merge(comps)
df.drop(columns='PELLET NAME', inplace=True)

# reorder
cols = list(df.columns[:3])
sort_cols = list(df.columns[3:])
sort_cols.sort()
cols.extend(sort_cols)
df = df[cols]

# add sol range label
range_names = []
ranges = np.arange(df.sol.min(), df.sol.max()+100, 100)
df.insert(2,'sol_range', None)
for i in np.arange(len(ranges)-1):
    min = ranges[i]
    max = ranges[i+1]
    name = f'{min}-{max}'
    range_names.append(name)
    
    df.loc[(df.sol>=min)&(df.sol<max), 'sol_range'] = name

## group by 100 sols

In [4]:
# reported RMSE
reported_rmse_dict = dict()
for oxide in comp_cols:
    avg = df[oxide+' RMSEP'].mean()
    std = df[oxide+' RMSEP'].std()
    
    reported_rmse_dict[oxide] = {
        'avg':avg,
        'std':std
    }

# actual RMSE
rmse_dict = dict()
for target in df['Target'].unique():
    temp = df[df.Target==target].copy()
    ct_results = []
    
    for s in temp.sol_range.values:
        sol_results = [s]
        
        temp_sol = temp[temp.sol_range==s].copy()
        # if only one measurement, just use squared error
        if len(temp_sol)==1:           
            for c in comp_cols:
                se = abs(temp_sol[c+'_actual'].values[0] - temp_sol[c].values[0])
                sol_results.append(se)
        
        else:
            for c in comp_cols:
                rmse = sqrt(mean_squared_error(temp_sol[c+'_actual'], temp_sol[c]))
                sol_results.append(rmse)
                
        # add to results for the target
        ct_results.append(sol_results)
        
    rmse_dict[target] = ct_results
    
# combine CT results
rmse_cols = [c+'_RMSE' for c in comp_cols]
rmse_cols.insert(0,'sol_range')
cts = list(df['Target'].unique())

for ct in cts:
    temp = pd.DataFrame(rmse_dict[ct], columns=rmse_cols)
    temp.insert(0,'Target',ct)
    
    if ct == cts[0]:
        rmse_results = temp.copy()
    else:
        rmse_results = pd.concat([rmse_results, temp], ignore_index=True)
rmse_results.drop_duplicates(inplace=True, ignore_index=True)
        
# summarize per sol
mean = rmse_results.groupby('sol_range', as_index=False).mean()
mean_cols = [c+'_mean' for c in rmse_cols[1:]]
mean_cols.insert(0,'sol_range')
mean.columns = mean_cols

std = rmse_results.groupby('sol_range', as_index=False).std()
std_cols = [c+'_std' for c in rmse_cols[1:]]
std_cols.insert(0,'sol_range')
std.columns = (std_cols)

rmse_summary = mean.merge(std)

# reorder
new_cols = list(rmse_summary.columns[1:])
new_cols.sort()
new_cols.insert(0,'sol_range')
rmse_summary.columns = new_cols

# add number of samples per group
n_samples = rmse_results.groupby('sol_range', as_index=False).size()
n_samples.rename(columns={'size':'n_samples'}, inplace=True)
rmse_summary = n_samples.merge(rmse_summary)

# add in empty ranges
empty_ranges = list(set(range_names) - set(rmse_summary.sol_range.unique()))
empty_range_df = pd.DataFrame([
    [empty_ranges[0], 0] + [np.nan]*len(rmse_summary.columns[2:]),
    [empty_ranges[1], 0] + [np.nan]*len(rmse_summary.columns[2:])
],
    columns = rmse_summary.columns
)
rmse_summary = pd.concat([rmse_summary, empty_range_df])

# sort for plotting
rmse_summary['min_sol'] = [int(x.split('-')[0]) for x in rmse_summary.sol_range]
rmse_summary.sort_values('min_sol', ignore_index=True, inplace=True)

# export
rmse_summary.to_csv('ChemCam_RMSE_bysol_summary.csv', index=False)
rmse_results.to_csv('ChemCam_RMSE_bysol.csv', index=False)

#### bar plot

In [ ]:
size = 10

x_labels = list(ranges)
x = np.arange(len(x_labels)-1)
x_ticks = xticks = [x-.5 for x in np.arange(len(x)+1)]

fig, ax = plt.subplots(nrows=3, 
                       ncols=3,
                       figsize = (15,10),
                       sharex=True)

x_ax = [0,1,2,0,1,2,0,1,2]
y_ax = [0,0,0,1,1,1,2,2,2]
i = 0
for oxide in comp_cols:
    
    # values
    yvals = rmse_summary[oxide+'_RMSE_mean']
    yerr = rmse_summary[oxide+'_RMSE_std']
    n_stds = rmse_summary['n_samples']
    avg_report = round(reported_rmse_dict[oxide]['avg'],2)
    avg_report_std = round(reported_rmse_dict[oxide]['std'],2)

    # reported RMSE box
    offset = avg_report_std/2
    origin = -.5
    y = avg_report - offset
    length = len(x)
    height = avg_report_std
    reported_errorbar = PatchCollection([Rectangle((origin, y), length, height)],
                                        edgecolor=None, facecolor='darkred', alpha=0.7)

    # actual error
    bar = ax[x_ax[i],y_ax[i]].bar(x, yvals, color='grey', zorder=0)
    
    # uncertainty
    ax[x_ax[i],y_ax[i]].errorbar(x, yvals, yerr=yerr, capsize=2, linestyle="", color='black', alpha=0.5)
    
    # number of standards
#    if i==0:
    ax[x_ax[i],y_ax[i]].bar_label(bar, n_stds)
    
    # reported error
    ax[x_ax[i],y_ax[i]].add_collection(reported_errorbar)
    
    ax[x_ax[i],y_ax[i]].set_title(f'Reported: {avg_report} +/- {avg_report_std}    Actual: {round(yvals.mean(),2)} +/- {round(yvals.std(),2)}',
                fontsize=size-1)
    
    # set axes
    ymin, ymax = ax[x_ax[i],y_ax[i]].get_ylim()
    ymin = 0
    ylimit = ceil(y+offset)
    if ymax < ylimit:
        ymax = ylimit
    ax[x_ax[i],y_ax[i]].set_ylim((ymin,ymax))
    
    # labels and ticks
    ax[x_ax[i],y_ax[i]].set_ylabel(f'{oxide} (wt%)', fontsize=size+4)
    ax[x_ax[i],y_ax[i]].set_xticks(x_ticks)
    ax[x_ax[i],y_ax[i]].set_xticklabels(labels=x_labels, fontsize=size-2, rotation=70)
    
    i+=1

#plt.annotate('Numbers above bars signify number of caltargets analysed during that time period',
#            (.1, .9), xycoords='figure fraction')
ax[2,1].set_xlabel('Sol (range limits)', fontsize=size+4)
plt.tight_layout()
plt.savefig('CCCT_rmsep_over_time_bar_plot.jpg', dpi=600)
plt.savefig('CCCT_rmsep_over_time_bar_plot.eps', dpi=600)
plt.show()

#### scatter plot

In [ ]:
size = 10

x_labels = list(ranges)
x = np.arange(len(x_labels)-1)
x_ticks = xticks = [x-.5 for x in np.arange(len(x)+1)]

fig, ax = plt.subplots(nrows=3, 
                       ncols=3,
                       figsize = (15,10),
                       sharex=True)

x_ax = [0,1,2,0,1,2,0,1,2]
y_ax = [0,0,0,1,1,1,2,2,2]
i = 0
for oxide in comp_cols:
    
    # values
    yvals = rmse_summary[oxide+'_RMSE_mean']
    yerr = rmse_summary[oxide+'_RMSE_std']
    n_stds = rmse_summary['n_samples']
    avg_report = round(reported_rmse_dict[oxide]['avg'],2)
    avg_report_std = round(reported_rmse_dict[oxide]['std'],2)

    # reported RMSE box
    offset = avg_report_std/2
    origin = -.5
    y = avg_report - offset
    length = len(x)
    height = avg_report_std
    reported_errorbar = PatchCollection([Rectangle((origin, y), length, height)],
                                        edgecolor=None, facecolor='darkred', alpha=0.6)
    ax[x_ax[i],y_ax[i]].add_collection(reported_errorbar)

    # points
    ax[x_ax[i],y_ax[i]].scatter(x, yvals, color='black', edgecolor=None, zorder=2)
    
    # errorbar
    ax[x_ax[i],y_ax[i]].errorbar(x, yvals, yerr=yerr, capsize=2, linestyle="", color='black', alpha=0.5, zorder=0)
    ax[x_ax[i],y_ax[i]].set_xlim((0.5, 29.5))
    
    # set axes
    ymin, ymax = ax[x_ax[i],y_ax[i]].get_ylim()
    if ymin <0:
        ymin=-0.25
    else:
        ymin = 0
    ylimit = ceil(y+offset)
    if ymax < ylimit:
        ymax = ylimit
    ax[x_ax[i],y_ax[i]].set_ylim((ymin,ymax))
    
    # labels and ticks
    ax[x_ax[i],y_ax[i]].set_title(f'Reported: {avg_report} +/- {avg_report_std}    Actual: {round(yvals.mean(),2)} +/- {round(yvals.std(),2)}',
            fontsize=size-1)
    ax[x_ax[i],y_ax[i]].set_ylabel(f'{oxide} (wt%)', fontsize=size+4)
    ax[x_ax[i],y_ax[i]].set_xticks(x_ticks)
    ax[x_ax[i],y_ax[i]].set_xticklabels(labels=x_labels, fontsize=size-2, rotation=70)
    
    i+=1

#plt.annotate('Numbers above bars signify number of caltargets analysed during that time period',
#            (.1, .9), xycoords='figure fraction')
ax[2,1].set_xlabel('Sol (range limits)', fontsize=size+4)
plt.tight_layout()
plt.savefig('CCCT_rmsep_over_time_scatter_plot.jpg', dpi=600)
plt.savefig('CCCT_rmsep_over_time_scatter_plot.eps', dpi=600)
plt.show()

### shergottite over time

In [44]:
sherg = df[df.Target=='Shergottite'].copy()

sherg_mean = sherg.groupby('sol_range', as_index=False)[comp_cols].mean()
mean_cols = [c+'_mean' for c in comp_cols]
mean_cols.insert(0,'sol_range')
sherg_mean.columns = mean_cols

sherg_std = sherg.groupby('sol_range', as_index=False)[comp_cols].std()
std_cols = [c+'_std' for c in comp_cols]
std_cols.insert(0,'sol_range')
sherg_std.columns = std_cols

sherg_summary = sherg_mean.merge(sherg_std)

# add number of samples per group
n_analyses = sherg.groupby('sol_range', as_index=False).size()
n_analyses.rename(columns={'size':'n_analyses'}, inplace=True)
sherg_summary = n_analyses.merge(sherg_summary)

# add in empty ranges
empty_ranges = list(set(range_names) - set(sherg_summary.sol_range.unique()))
empty_range_df = pd.DataFrame([
    [empty_ranges[0], 0] + [np.nan]*len(sherg_summary.columns[2:]),
    [empty_ranges[1], 0] + [np.nan]*len(sherg_summary.columns[2:]),
    [empty_ranges[2], 0] + [np.nan]*len(sherg_summary.columns[2:])
],
    columns = sherg_summary.columns
)
sherg_summary = pd.concat([sherg_summary, empty_range_df])

# sort for plotting
sherg_summary['min_sol'] = [int(x.split('-')[0]) for x in sherg_summary.sol_range]
sherg_summary.sort_values('min_sol', ignore_index=True, inplace=True)

#### bar plot

In [ ]:
x_labels = list(ranges)
x = np.arange(len(x_labels)-1)
x_ticks = xticks = [x-.5 for x in np.arange(len(x)+1)]

fig, ax = plt.subplots(nrows=3, 
                       ncols=3,
                       figsize = (15,10),
                       sharex=True)

x_ax = [0,1,2,0,1,2,0,1,2]
y_ax = [0,0,0,1,1,1,2,2,2]
i = 0
for oxide in comp_cols:
    
    # values
    yvals = sherg_summary[oxide+'_mean']
    yerr = sherg_summary[oxide+'_std']
    n_stds = sherg_summary['n_analyses']
    actual = sherg[oxide+'_actual'].values[0]

    # actual error
    bar = ax[x_ax[i],y_ax[i]].bar(x, yvals, color='grey', zorder=0)
    
    # uncertainty
    ax[x_ax[i],y_ax[i]].errorbar(x, yvals, yerr=yerr, capsize=2, linestyle="", color='black', alpha=0.5)
    
    # number of standards
#    if i==0:
    ax[x_ax[i],y_ax[i]].bar_label(bar, n_stds)
    
    # reported error
    ax[x_ax[i],y_ax[i]].axhline(actual, color='darkred')
    ax[x_ax[i],y_ax[i]].set_title(f'Reported: {avg_report} +/- {avg_report_std}    Actual: {round(yvals.mean(),2)} +/- {round(yvals.std(),2)}',
                fontsize=size-1)
    
    # y limits
    ymin, ymax = ax[x_ax[i],y_ax[i]].get_ylim()
    ax[x_ax[i],y_ax[i]].set_ylim(0, ymax)
    
    # labels and ticks
    ax[x_ax[i],y_ax[i]].set_ylabel(f'{oxide} (wt%)', fontsize=size+4)
    ax[x_ax[i],y_ax[i]].set_xticks(x_ticks)
    ax[x_ax[i],y_ax[i]].set_xticklabels(labels=x_labels, fontsize=size-2, rotation=70)
    
    i+=1

#plt.annotate('Numbers above bars signify number of caltargets analysed during that time period',
#            (.1, .9), xycoords='figure fraction')
ax[2,1].set_xlabel('Sol (range limits)', fontsize=size+4)
plt.title('Shergottite', fontsize=size+4)
plt.tight_layout()
plt.savefig('shergottite_over_time_bar_plot.jpg', dpi=600)
plt.savefig('shergottite_over_time_bar_plot.eps', dpi=600)
plt.show()

## by sample
Get average prediction and stdev of that for pred/true

In [29]:
# number of duplicates
dup_df = pd.DataFrame(df.groupby('Target').size()).rename(columns={0:'n_dups'})

# average value
mean_df = df.groupby('Target')[comp_cols].mean()
mean_df.columns = [c+'_mean' for c in comp_cols]

# stdev of values
stdev_df = df.groupby('Target')[comp_cols].std()
stdev_df.columns = [c+'_std' for c in comp_cols]

# actual values
actual_df = df[[x for x in df.columns if ('Target' in x)|('actual' in x)]].drop_duplicates().set_index('Target')

# merge results
results_df = dup_df.join(mean_df).join(stdev_df).join(actual_df).reset_index()

# sort columns
all_cols = list(results_df.columns[2:])
all_cols.sort()
all_cols.insert(0,'Target')
all_cols.insert(1,'n_dups')
results_df = results_df[all_cols]

results_df.to_csv('ChemCam_pred_true_summary.csv', index=False)